In [1]:
import os

import pandas as pd
import tensorflow.contrib.keras as keras

from vixstructure.utils import parse_model_repr
from vixstructure.data import LongPricesDataset

## Experiment 01

In [2]:
dataset = LongPricesDataset("data/8_m_settle.csv", "data/expirations.csv")
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset.splitted_dataset(normalize=True)
directory, _, files = next(os.walk("models/experiment01"))
parameters = [parse_model_repr(os.path.splitext(file)[0]) for file in files if os.path.splitext(file)[1] == ".h5"]
# If you give the load_models function custom objects loading takes a lot longer. :(
models = zip((keras.models.load_model(os.path.join(directory, file), custom_objects={"denorm_mse":dataset.denorm_mse})
              for file in files), parameters)

In [11]:
model = next(models)

In [13]:
# History isn't included. Fuck!
model = model[0].history

AttributeError: 'Model' object has no attribute 'history'